In [69]:
from src.utils import read_config
sys.path.append('/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction')
sys.path.append('/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction/src')

from src.data.make_dataset import *

In [9]:
config_file = read_config("/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction/params.yaml")

In [6]:
import yaml
yaml.safe_load(open("params.yaml"))

{'base': {'label_mapper': {'-1': '0,',
   '0': '1,',
   '1': '1,',
   '2': '1,',
   '3': '1,',
   '4': '1,',
   '5': 1},
  'project': 'credit_card_approval_prediction',
  'random_state': 42}}

In [11]:
file = "/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction/params.yaml"
create_interm_data(file)

[Errno 2] No such file or directory: 'data/raw/application_record.csv'


In [77]:
import pandas as pd
import os
print(os.getcwd())
os.chdir("/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction")
df = pd.read_csv("data/interim/joined_data.csv")

/Users/goldyrana/work/ATU/Sem1/predictive/t_project1/credit_card_approval_prediction


In [78]:
import argparse
import os, yaml, joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from src.utils import read_config
# from src.utils import encoders, custom_pd,  logging
from src.utils import *
from dataclasses import dataclass

In [79]:
config_file = read_config("params.yaml")

@dataclass
class meta_data:
    # data path
    application_record = config_file["raw_data_source"]["application_record"]
    credit_record = config_file["raw_data_source"]["credit_record"]
    interim_data_path = config_file["data_source"]["interim_data"]
    
    # features manipulation
    drop_col = config_file["drop_columns"]
    greater_than = config_file["base"]["second_label_mapper"]["greater_than"]
    less_than = config_file["base"]["second_label_mapper"]["less_than"]
    target_col = config_file["base"]["target_col"]

    # label mapper
    mapper = config_file["base"]["label_mapper"]
    mapper_agg_function = config_file["base"]["mapper_agg_function"]
 



In [80]:
for key, value in meta_data.mapper.items(): # loop to convert the values in the dictionary to int defined in params.yaml
    meta_data.mapper[key] = int(value)

In [81]:
meta_data.mapper

{'-1': 0, '0': 1, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, 'C': -1, 'X': 0}

In [82]:
df.shape

(438557, 20)

In [146]:
df1 = pd.read_csv(meta_data.application_record)
df2 = pd.read_csv(meta_data.credit_record)

In [147]:
def create_label(data: pd.DataFrame, column: str, mapper: dict):
    """Function is called to create labels for the data
    """
    data[column] = data[column].map(mapper)
    return data

# data = create_label(df, meta_data.target_col, meta_data.mapper)

In [148]:
df1.isnull().sum()  

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [149]:
df1.shape

(438557, 18)

In [150]:
# vintage analysis function 
data = df2
for key, value in meta_data.mapper.items(): # loop to convert the values in the dictionary to int defined in params.yaml
    meta_data.mapper[key] = int(value)

In [151]:
data.head(2)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0


In [152]:
def lower_name(data: pd.DataFrame):
    try:
        data.columns = [column.lower() for column in data.columns]
    except Exception as e:
        logging.warning(f"Lowering the column names failed... [{e}]")
    return data

df1 = lower_name(df1)
df2 = lower_name(df2)


In [153]:
df2.head(3)

,id,months_balance,status
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0


In [154]:
data = df2

In [155]:
data = create_label(data, meta_data.target_col, meta_data.mapper)     

In [156]:
data.isnull().sum() 

id                0
months_balance    0
status            0
dtype: int64

In [157]:
data

,id,months_balance,status
0,5001711,0,0
1,5001711,-1,1
2,5001711,-2,1
3,5001711,-3,1
4,5001712,0,-1
...,...,...,...
1048570,5150487,-25,-1
1048571,5150487,-26,-1
1048572,5150487,-27,-1
1048573,5150487,-28,-1


In [158]:
data_with_label = data.groupby('id').agg({meta_data.target_col: meta_data.mapper_agg_function})

In [159]:
data_with_label.reset_index(inplace=True)


In [160]:
data_with_label

,id,status
0,5001711,3
1,5001712,1
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,6
45981,5150483,0
45982,5150484,11
45983,5150485,2


In [161]:


def second_label_mapper(x):
    if int(meta_data.less_than) == int(meta_data.greater_than):
        logging.error("Greater than and less than values are same")
    if x <= int(meta_data.less_than):
        return 0 # Good client
    elif x >= int(meta_data.greater_than):
        return 1 # Bad client 
    else:
        logging.info("Error in creating labels for the target column: Vintage analysis")
        
data_with_label[meta_data.target_col] = data_with_label[meta_data.target_col].apply(lambda x: second_label_mapper(x))


In [162]:
data_with_label.isnull().sum()

id        0
status    0
dtype: int64

In [163]:
data_with_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45985 entries, 0 to 45984
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      45985 non-null  int64
 1   status  45985 non-null  int64
dtypes: int64(2)
memory usage: 718.6 KB


In [164]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   438557 non-null  int64  
 1   code_gender          438557 non-null  object 
 2   flag_own_car         438557 non-null  object 
 3   flag_own_realty      438557 non-null  object 
 4   cnt_children         438557 non-null  int64  
 5   amt_income_total     438557 non-null  float64
 6   name_income_type     438557 non-null  object 
 7   name_education_type  438557 non-null  object 
 8   name_family_status   438557 non-null  object 
 9   name_housing_type    438557 non-null  object 
 10  days_birth           438557 non-null  int64  
 11  days_employed        438557 non-null  int64  
 12  flag_mobil           438557 non-null  int64  
 13  flag_work_phone      438557 non-null  int64  
 14  flag_phone           438557 non-null  int64  
 15  flag_email       

In [166]:
data_with_label

,id,status
0,5001711,1
1,5001712,1
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,1
45981,5150483,0
45982,5150484,1
45983,5150485,1


In [141]:
df1[df1.id == 5150483]

,id,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members
12886,5150483,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0


In [167]:
df1.shape

(438557, 18)

In [196]:
my_series = pd.Series([1, 2, 3, 4, 5])

# Checking the data type
data_type_of_series = my_series.dtype

print("Data type of the Series:", data_type_of_series)

Data type of the Series: int64


In [173]:
data_with_label.shape

(45985, 2)

In [170]:
x.shape

(438557, 20)

In [174]:
print(df1["id"].dtype)
print(data_with_label["id"].dtype)

int64
int64


In [175]:
common_ids = set(df1["id"]).intersection(set(data_with_label["id"]))


In [177]:
df1.columns

Index(['id', 'code_gender', 'flag_own_car', 'flag_own_realty', 'cnt_children',
       'amt_income_total', 'name_income_type', 'name_education_type',
       'name_family_status', 'name_housing_type', 'days_birth',
       'days_employed', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members'],
      dtype='object')

In [178]:
data_with_label.columns

Index(['id', 'status'], dtype='object')

In [181]:
x = df1.join(data_with_label, on = "id", how = "left", lsuffix="main")
x.isnull().sum()/len(df1)

idmain                 0.00000
code_gender            0.00000
flag_own_car           0.00000
flag_own_realty        0.00000
cnt_children           0.00000
amt_income_total       0.00000
name_income_type       0.00000
name_education_type    0.00000
name_family_status     0.00000
name_housing_type      0.00000
days_birth             0.00000
days_employed          0.00000
flag_mobil             0.00000
flag_work_phone        0.00000
flag_phone             0.00000
flag_email             0.00000
occupation_type        0.30601
cnt_fam_members        0.00000
id                     1.00000
status                 1.00000
dtype: float64

In [185]:
data_with_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45985 entries, 0 to 45984
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      45985 non-null  int64
 1   status  45985 non-null  int64
dtypes: int64(2)
memory usage: 718.6 KB


In [188]:
df1.shape

(438557, 18)

In [193]:
d = df1.join(data_with_label.set_index("id"), on = "id", how = "left", rsuffix="_main")

In [195]:
d.isnull().sum()/len(df1)

id                     0.000000
code_gender            0.000000
flag_own_car           0.000000
flag_own_realty        0.000000
cnt_children           0.000000
amt_income_total       0.000000
name_income_type       0.000000
name_education_type    0.000000
name_family_status     0.000000
name_housing_type      0.000000
days_birth             0.000000
days_employed          0.000000
flag_mobil             0.000000
flag_work_phone        0.000000
flag_phone             0.000000
flag_email             0.000000
occupation_type        0.306010
cnt_fam_members        0.000000
status                 0.916871
dtype: float64

In [190]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   438557 non-null  int64  
 1   code_gender          438557 non-null  object 
 2   flag_own_car         438557 non-null  object 
 3   flag_own_realty      438557 non-null  object 
 4   cnt_children         438557 non-null  int64  
 5   amt_income_total     438557 non-null  float64
 6   name_income_type     438557 non-null  object 
 7   name_education_type  438557 non-null  object 
 8   name_family_status   438557 non-null  object 
 9   name_housing_type    438557 non-null  object 
 10  days_birth           438557 non-null  int64  
 11  days_employed        438557 non-null  int64  
 12  flag_mobil           438557 non-null  int64  
 13  flag_work_phone      438557 non-null  int64  
 14  flag_phone           438557 non-null  int64  
 15  flag_email       

In [183]:
x = data_with_label.join(df1, on = "id", how = "right", lsuffix="main")
x.isnull().sum()/len(df1)

idmain                 1.00000
status                 1.00000
id                     0.00000
code_gender            0.00000
flag_own_car           0.00000
flag_own_realty        0.00000
cnt_children           0.00000
amt_income_total       0.00000
name_income_type       0.00000
name_education_type    0.00000
name_family_status     0.00000
name_housing_type      0.00000
days_birth             0.00000
days_employed          0.00000
flag_mobil             0.00000
flag_work_phone        0.00000
flag_phone             0.00000
flag_email             0.00000
occupation_type        0.30601
cnt_fam_members        0.00000
dtype: float64

In [116]:
x.shape

(438557, 22)

In [50]:
df.isnull().sum()

idmain                      0
code_gender                 0
flag_own_car                0
flag_own_realty             0
cnt_children                0
amt_income_total            0
name_income_type            0
name_education_type         0
name_family_status          0
name_housing_type           0
days_birth                  0
days_employed               0
flag_mobil                  0
flag_work_phone             0
flag_phone                  0
flag_email                  0
occupation_type        134203
cnt_fam_members             0
id                     438557
status                 438557
dtype: int64

In [51]:
df.head()

,idmain,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,id,status
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,NaN,NaN
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,NaN,NaN
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,NaN,NaN
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,NaN,NaN
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,NaN,NaN


In [21]:
df.shape

(438557, 20)

In [40]:
def seprate_null(data: pd.DataFrame, with_column: str):
    """Function is called to seprate the null values
    """
    try:
        null_data = data[data[with_column].isnull()]
        not_null_data = data[data[with_column].notnull()]
        logging.info("Null values seprated sucessfully...")
        return null_data, not_null_data
    except Exception as e:
        logging.error(f"Error in seprating null values... [{e}]")

n, nn = seprate_null(df, 'status')

In [45]:
n[n.status.notnull()]

,idmain,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,id,status


In [47]:
n.shape

(438557, 20)

In [48]:
df.shape

(438557, 20)

In [46]:
n.isnull().sum()

idmain                      0
code_gender                 0
flag_own_car                0
flag_own_realty             0
cnt_children                0
amt_income_total            0
name_income_type            0
name_education_type         0
name_family_status          0
name_housing_type           0
days_birth                  0
days_employed               0
flag_mobil                  0
flag_work_phone             0
flag_phone                  0
flag_email                  0
occupation_type        134203
cnt_fam_members             0
id                     438557
status                 438557
dtype: int64

In [43]:
nn.head()

,idmain,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,id,status


In [37]:
x = pd.read_csv("data/interim/null_data.csv")

In [38]:
x.isnull().sum()

idmain                   0
flag_own_realty          0
cnt_children             0
amt_income_total         0
name_income_type         0
days_birth               0
days_employed            0
flag_mobil               0
flag_work_phone          0
flag_phone               0
flag_email               0
occupation_type     134203
cnt_fam_members          0
id                  438557
status              438557
dtype: int64

In [36]:
x.head()

,idmain,flag_own_realty,cnt_children,amt_income_total,name_income_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,id,status
